In [1]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras import optimizers
from keras import layers
from keras import models
from keras import losses
import skimage.io as io
import pandas as pd
import numpy as np
import glob
import cv2

# Importing all necessary data
x_train = [io.imread(file) for file in glob.glob("../resized/*.png")]
y_train = pd.read_csv("./histeq/train_labels.csv")

x_test = [io.imread(file) for file in glob.glob("../resizedtest/*.png")]
y_test = pd.read_csv("./histeq/test_labels.csv")

y_train = y_train["Disease_Risk"]
y_test = y_test["Disease_Risk"]

x_graph_train = [io.imread(file, as_gray=True) for file in glob.glob("../resizedtraingraph/*.png")]
y_graph_train = y_train.iloc[:1427]

x_graph_test = [io.imread(file, as_gray=True) for file in glob.glob("../resizedtestgraph/*.png")]
y_graph_test = y_test

# Converting data to ndarrays
x_train = np.array(x_train)
x_graph_train = np.array(x_graph_train)

x_test = np.array(x_test)
x_graph_test = np.array(x_graph_test)

y_train = np.asarray(y_train)
y_graph_train = np.asarray(y_graph_train)

# Concatenating the graph and the image data
x_train = np.concatenate((x_train, x_graph_train), axis=0)
y_train = np.concatenate((y_train, y_graph_train), axis=0)

x_test = np.concatenate((x_test, x_graph_test), axis=0)
y_test = np.concatenate((y_test, y_graph_test), axis=0)

# Normalizing the train values
for i in range(len(x_train)):
    x_train[i] = x_train[i].astype("float32")
    x_train[i] = x_train[i] / 255
    
# Building the model
model = Sequential()

model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape=(178, 268, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = (64 ,64,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation = 'relu', input_shape = (64 ,64,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(units = 512, activation = 'relu'))

model.add(layers.Dense(units = 1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics = ['acc'])

# Based on the trainings, optimal number of epochs is 7.
model.fit(x_train, y_train, epochs = 7)

# Evaluating the model
results_test = model.evaluate(x_test, y_test)
print(results_test)

model.save('cnn_model.h5')

Epoch 1/7


2023-01-04 14:50:43.949900: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-04 14:50:44.346682: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 24s 216ms/step - loss: 0.5134 - acc: 0.8127
Epoch 2/7
105/105 [==============================] - 22s 213ms/step - loss: 0.4833 - acc: 0.8127
Epoch 3/7
105/105 [==============================] - 22s 214ms/step - loss: 0.4894 - acc: 0.8127
Epoch 4/7
105/105 [==============================] - 22s 213ms/step - loss: 0.4832 - acc: 0.8127
Epoch 5/7
105/105 [==============================] - 23s 216ms/step - loss: 0.4804 - acc: 0.8127
Epoch 6/7
105/105 [==============================] - 22s 214ms/step - loss: 0.4817 - acc: 0.8127
Epoch 7/7
105/105 [==============================] - 22s 213ms/step - loss: 0.4774 - acc: 0.8127


In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 176, 266, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 88, 133, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 86, 131, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 65, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 63, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 20, 31, 128)      0

2023-01-04 14:53:24.014423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


40/40 [==============================] - 4s 80ms/step - loss: 28.0025 - acc: 0.7906
[28.00250244140625, 0.7906250357627869]


In [21]:
model.save('cnn_model.h5')